In [ ]:
import openai
import json

with open('server/model/utils/config.json') as f:
    config = json.load(f)

openai.api_key = config['chatgpt']['secret']

## Token 수 세기

In [ ]:
from typing import List
import tiktoken


def counter(text : str):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    return len(encoding.encode(text))


def count_money(input : str,
                output : str,
                encoder: tiktoken.Encoding):
    # $0.0015 / 1K tokens input
    # $0.003 / 1K tokens output
    token_in = len(encoder.encode(input))
    price_in = token_in *  0.0015 / 1000
    token_out = len(encoder.encode(output))
    price_out = token_out * 0.003 / 1000
    total_price_dollar = price_in + price_out
    total_price_won = total_price_dollar * 1300
    return token_in, token_out, total_price_dollar, total_price_won


def count_money(in_tokens : int,
                out_tokens : int,
                ):
    # $0.0015 / 1K tokens input
    # $0.003 / 1K tokens output
    price_in = in_tokens * 0.0015 / 1000
    price_out = out_tokens * 0.003 / 1000
    total_price_dollar = price_in + price_out
    total_price_won = total_price_dollar * 1300
    return total_price_dollar, total_price_won


def count_money(texts: List[str],
                encoder : tiktoken.Encoding):
    # $0.0001 / 1K tokens
    token_counts = 0
    for text in texts:
        token_counts += len(encoder.encode(text))
    price = token_counts * 0.0001 / 1000
    price_won = price * 1300
    return token_counts, price, price_won

In [ ]:
def get_chat(
        user : str,
        system : str,
        assistant : str,
):
    prompt_message = [
        {"role": "system",
            "content": system},
        {"role": "assistant",
            "content": assistant},
        {"role": "user",
            "content": user
        }
    ]
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=prompt_message,
            presence_penalty=0.6
        )
    except:
        print("OpenAI Response Error")

    chat = response['choices'][0]['message']['content']
    
    token_in, token_out, total_price_dollar, total_price_won = count_money(user, chat, tiktoken.encoding_for_model("gpt-3.5-turbo"))
    return token_in, token_out, total_price_dollar, total_price_won, chat

In [ ]:
def get_embedding(texts: List[str], encoder: tiktoken.Encoding):
    token_counts, price_dollar, price_won = count_money(texts, encoder)
    result = openai.Embedding.create(
        engine="text-embedding-ada-002",
        input=texts)
    return result["data"][0]["embedding"], token_counts, price_won